<a href="https://colab.research.google.com/github/haru0l/Diff-SVC-notebooks/blob/main/Diff_SVC_training_notebook_(colab_ver_).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Training notebook for [Diff-SVC](https://github.com/prophesier/diff-svc) written by [Nekro](https://twitter.com/NekroTheCorpse) of [Archivoice](https://github.com/archivoice)


# Check Setup

In [ ]:
#@title #Check GPU type
#@markdown this is for checking the GPU type you have as well as the available amount of vram.
!nvidia-smi -L
!nvidia-smi

In [ ]:
#@title #Mount Google Drive

#@markdown Makes your life easier when uploading and saving stuff.

from google.colab import drive
drive.flush_and_unmount()
!rm -rf /content/drive
drive.mount('/content/drive')
print('Done!')

# Preparation

In [ ]:
#@title #Step 1: Install Diff-SVC
#@markdown The stuff you'll need for every other thing afterwards.

import os
print('Upgrading pip & installing 7zip')
!rm -rf /content/sample_data
!python -m pip install --upgrade pip
!python -m pip install --upgrade wheel
!apt-get install unzip
!pip install gdown

print('Installing torch')
%pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install --pre torchtext==0.6.0 --no-deps

print('Installing Diff-SVC')
!git clone https://github.com/prophesier/diff-svc &> /dev/null
%cd 

print('Installing requirements')
%cd "/content/diff-svc/"
!pip install -r requirements_short.txt
!pip install tensorboard<2.9,>=2.8
%reload_ext tensorboard

%cd "/content/diff-svc/training/"
!rm config.yaml
!gdown 'https://drive.google.com/uc?id=1FeYxQZI-n-_GLPktq1aEVzXH0IM7_i3F' -O config.yaml
%cd "/content/"
!wget "https://github.com/haru0l/Diff-SVC-notebooks/releases/download/checkpoints/checkpoints.zip" -O checkpoints.zip
%mkdir -p /content/diff-svc/checkpoints/
!unzip /content/checkpoints.zip -d /content/diff-svc/

print('Done!')

In [ ]:
#@title #Step 2: Decompress dataset
#@markdown This should work with most common archive formats, so don't worry.

#@markdown Supported types: `.rar`, `.zip`, `.tar`, `.tar.gz`, `.tar.bz2`, `.7z`

#@markdown ###Note that your dataset should consist of `.wav` or `.ogg` format audio
#@markdown ---
#@markdown Name your singer.
singer_name = 'Unnamed' #@param {type: "string"}
%cd "/content/"
!sed -i -r 's/atri/{singer_name}/g' /content/diff-svc/training/config.yaml

#@markdown ---
#@markdown File location
!mkdir -p /content/diff-svc/data/raw
dataset_location = '/content/drive/MyDrive/*' #@param {type: "string"}
diffsvc_location = os.path.join('diff-svc', 'data', 'raw', singer_name, "")

if dataset_location.endswith('.rar'):
    !unrar x "$dataset_location" "$diffsvc_location"
elif dataset_location.endswith('.zip'):
    !unzip "$dataset_location" -d "$diffsvc_location"
elif dataset_location.endswith('.tar'):
    !tar -xf "$dataset_location" -C "$diffsvc_location"
elif dataset_location.endswith('.tar.gz'):
    !tar -xzf "$dataset_location" -C "$diffsvc_location"
elif dataset_location.endswith('.tar.bz2'):
    !tar -xjf "$dataset_location" -C "$diffsvc_location"
else:
    !7za x "$dataset_location" -o$diffsvc_location

print('Done!')

In [ ]:
#@title #Step 2-A: Decompress training data
#@markdown Decompresses training data directly to `diff-svc/data/binary`, usable only if you already have the output files of Step 4.

#@markdown Supported types: check above. 

#@markdown ###You must match the training settings of the config.yaml file you used to generated the preprocessed data.
#@markdown ---
#@markdown Name your singer.
singer_name = 'Unnamed' #@param {type: "string"}
!sed -i -r 's/atri/{singer_name}/g' /content/diff-svc/training/config.yaml

#@markdown ---
#@markdown File location
!mkdir -p /content/diff-svc/data/binary
preprocessed_data_location = '/content/drive/MyDrive/*' #@param {type: "string"}
diffsvc_bin_location = '/content/diff-svc/data'

if preprocessed_data_location.endswith('.rar'):
    !unrar x "$preprocessed_data_location" "$diffsvc_bin_location"
elif preprocessed_data_location.endswith('.zip'):
    !unzip "$preprocessed_data_location" -d "$diffsvc_bin_location"
elif preprocessed_data_location.endswith('.tar'):
    !tar -xf "$preprocessed_data_location" -C "$diffsvc_bin_location"
elif preprocessed_data_location.endswith('.tar.gz'):
    !tar -xzf "$preprocessed_data_location" -C "$diffsvc_bin_location"
elif preprocessed_data_location.endswith('.tar.bz2'):
    !tar -xjf "$preprocessed_data_location" -C "$diffsvc_bin_location"
else:
    !7za x "$preprocessed_data_location" -o$diffsvc_bin_location

print('Done!')

# Training Options/Parameters
Unfortunately, you can not get away with no editing, not completely that is.

In [ ]:
#@title #Step 3: Edit training parameters

#@markdown ---
#@markdown ###Training step count
#@markdown Number of steps during the diffusion process, editing this is not recommnded
K_step = 1000 #@param {type: "integer"}

#@markdown ---
#@markdown ###Optimize training to GPU
#@markdown Different GPU's have different amounts of vram, check this if you have a Tesla V100 or A100.
v100_or_a100 = False #@param {type: "boolean"}
if v100_or_a100:
    batch_size = 50
else:
    batch_size = 12

#@markdown ---
#@markdown ###F0 extraction method
#@markdown Crepe is used for F0 extraction for data preprocessing, while it is of higher quality, it is slow, therefore set to false as default.
use_crepe = True #@param {type: "boolean"}

#@markdown ---
#@markdown ###Set checkpoint interval
#@markdown As the name states, saves a checkpoint at an interval. When using GPU training, it runs quite fast, so try not to touch this, there's no point.
checkpoint_interval = 1000 #@param {type: "integer"}

decay_steps = 20000

#@markdown ---
#@markdown ###Use pretrained model
#@markdown Use a pretrained model for better quality with lower amounts of data.
use_pretrained_model = True #@param {type: "boolean"}

#@markdown ---
#@markdown ###Use custom pretrained model
#@markdown Use a pretrained model of you own choice that we don't provide. 
pretrained_model_directory = 'Insert directory here' #@param {type: "string"}

#@markdown ---
#@markdown ###Use custom save directory
#@markdown You can change the directory to save wherever you want. Default location is /diff-svc/checkpoint if unchanged.
save_dir = 'Insert directory here' #@param {type: "string"}

if (save_dir != "Insert directory here") or (save_dir != ""):
  %cd /content/diff-svc/utils
  !rm -rf hparams.py
  !wget https://github.com/prophesier/diff-svc/raw/main/utils/hparams.py
  !sed -i -r 's|checkpoints/\{args.work_dir}|haruwashere|g' /content/diff-svc/utils/hparams.py
  !sed -i -r 's|haruwashere|{save_dir}|g' /content/diff-svc/utils/hparams.py
  %cd /content/
else:
  %cd /content/diff-svc/utils
  !rm -rf hparams.py
  !wget https://github.com/prophesier/diff-svc/raw/main/utils/hparams.py
  %cd /content/


if use_pretrained_model:
  pretrained_model_directory = '/content/diff-svc/pretrain/nyaru/model_ckpt_steps_60000.ckpt'
  !wget "https://github.com/haru0l/Diff-SVC-notebooks/releases/download/checkpoints/nyaru.zip" -O nyaru.zip
  %mkdir -p /content/diff-svc/pretrain/
  !unzip /content/nyaru.zip -d /content/diff-svc/pretrain/
  lr = '0.00005'
else:
  lr = '0.0008'

!sed -i -r 's|(decay_steps:)(\s+)(.+)|\1\2{decay_steps}|g' /content/diff-svc/training/config.yaml
!sed -i -r 's|(lr:)(\s+)(.+)|\1\2{lr}|g' /content/diff-svc/training/config.yaml
!sed -i -r 's|(K_step:)(\s+)(.+)|\1\2{K_step}|g' /content/diff-svc/training/config.yaml
!sed -i -r 's|(max_sentences:)(\s+)(.+)|\1\2{batch_size}|g' /content/diff-svc/training/config.yaml
!sed -i -r 's|(use_crepe:)(\s+)(.+)|\1\2{use_crepe}|g' /content/diff-svc/training/config.yaml
!sed -i -r 's|(val_check_interval:)(\s+)(.+)|\1\2{checkpoint_interval}|g' /content/diff-svc/training/config.yaml
!sed -i -r 's|(pretrain_fs_ckpt:)(\s+)(.+)|\1\2{pretrained_model_directory}|g' /content/diff-svc/training/config.yaml

# Training
Finally, the dreaded part.

In [ ]:
#@title #Step 4: Pre-processing
#@markdown This step is also known as data prep or feature generation, who cares?
#@markdown
%cd "/content/diff-svc/"

os.environ['PYTHONPATH']='.'
!CUDA_VISIBLE_DEVICES=0 python preprocessing/binarize.py --config training/config.yaml
%cd "/content/diff-svc/data"
!7za -bso0 a "/content/{singer_name}_binary_data.7z" "binary/{singer_name}"
if not os.path.exists('/content/drive/MyDrive/Diff-SVC/data'):
    !mkdir -p /content/drive/MyDrive/Diff-SVC/data
!mv -v "/content/{singer_name}_binary_data.7z" /content/drive/MyDrive/Diff-SVC/data

In [7]:
#@title #Step 5-0: Setup for small datasets
#@markdown If your dataset is small (around 30 minutes to 1 hour), each epoch will go by very fast and won't have enough time to train well, so if your dataset is considered small, run this cell.

#@markdown DO NOT RUN IF YOUR DATASET IS GOOD ENOUGH IN TERMS OF SIZE!

endless_ds = True
!sed -i -r 's|(endless_ds:)(\s+)(.+)|\1\2{endless_ds}|g' /content/diff-svc/training/config.yaml
print("done!")

done!


In [ ]:
#@title #Step 5-1 Tensorboard (run before step 5)
#@markdown Shows training progress, go to the top right corner to set it to update the logs.

import datetime, os

if save_dir != "Insert directory here":
  %tensorboard --load_fast=true --reload_interval=1 --reload_multifile=true --logdir="{save_dir}/lightning_logs/"
else:
  %tensorboard --load_fast=true --reload_interval=1 --reload_multifile=true --logdir=/content/diff-svc/checkpoints/{singer_name}/lightning_logs/


In [ ]:
#@title #Step 5: Training
#@markdown Yeah, it took THAT long to get here, colab is probably going to disconnect you at this point... unless you have pro ¯\\_(ツ)_/¯
%cd "/content/diff-svc/"

os.environ['PYTHONPATH']='.'

!CUDA_VISIBLE_DEVICES=0 python run.py --config training/config.yaml --exp_name $singer_name --reset

In [ ]:
#@title # Step 6: Package Model
store_on_drive = True #@param {type: "boolean"}
from datetime import datetime, timezone
import glob

time_now = datetime.now(timezone.utc).strftime('%Y-%m-%d %H-%M-%S')

archive_name = f'{singer_name}_{time_now}'

%cd /content/diff-svc/checkpoints
!zip -r "/content/{archive_name}.zip" ./{singer_name} -x ./{singer_name}/lighting_logs/\*

if store_on_drive:
    if not os.path.exists('/content/drive/MyDrive/Diff-SVC_release'):
        !mkdir /content/drive/MyDrive/Diff-SVC_release
    
    !mv -v "/content/{archive_name}.zip" /content/drive/MyDrive/Diff-SVC_release

#clear_output()
print('Done!')